<a href="https://colab.research.google.com/github/Face-Expression-Recognation-Team-2/.github/blob/main/Face_Emotion_Recognition_(_Team_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Memasukkan Library

In [2]:
import numpy as np
import pandas as pd
import os
import zipfile
import shutil
import cv2

#Memasukkan data

In [4]:
!pip install -q kaggle
! mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d 'jonathanoheix/face-expression-recognition-dataset'

mkdir: cannot create directory ‘/root/.kaggle’: File exists
100% 121M/121M [00:02<00:00, 95.8MB/s]
100% 121M/121M [00:02<00:00, 58.7MB/s]


In [5]:
dataset_zip = zipfile.ZipFile('face-expression-recognition-dataset.zip', 'r')

dataset_zip.extractall()

dataset_zip.close()

In [6]:
source_folders = ['/content/images/images/train', '/content/images/images/validation']
destination_folder = '/content/dataset'

if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

for folder in source_folders:
    shutil.move(folder, destination_folder)
    print(f"Folder {folder} berhasil dipindahkan ke {destination_folder}")

print("Success")

Folder /content/images/images/train berhasil dipindahkan ke /content/dataset
Folder /content/images/images/validation berhasil dipindahkan ke /content/dataset
Success


In [7]:
old_folder = '/content/images'

shutil.rmtree(old_folder)

print("Success")

Success


#Tahap Pengerjaan

In [8]:
# 1. mengambil hanya 50 gambar per emosi
dataset_folder = "dataset"
train_folder = os.path.join(dataset_folder, "train")
validation_folder = os.path.join(dataset_folder, "validation")

images_folder = "images"

emosi = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]

def copy_images(source_folder, destination_folder, emosi, num_images):
    for emosi in emosi:

        source_emotion_folder = os.path.join(source_folder, emosi)
        destination_emotion_folder = os.path.join(destination_folder, emosi)

        if not os.path.exists(destination_emotion_folder):
            os.makedirs(destination_emotion_folder)

        images = os.listdir(source_emotion_folder)[:num_images]

        for image in images:
            source_image_path = os.path.join(source_emotion_folder, image)
            destination_image_path = os.path.join(destination_emotion_folder, image)
            shutil.copyfile(source_image_path, destination_image_path)

if not os.path.exists(images_folder):
    os.makedirs(images_folder)

train_images_folder = os.path.join(images_folder, "train")
copy_images(train_folder, train_images_folder, emosi, 50)

validation_images_folder = os.path.join(images_folder, "validation")
copy_images(validation_folder, validation_images_folder, emosi, 50)

print("sukses")

sukses


In [9]:
old_folder = '/content/dataset'

shutil.rmtree(old_folder)

print("Success")

Success


In [10]:
# 2. preprocessing ( Peningkatan Kontras )

train_folder = "images/train"
validation_folder = "images/validation"

def peningkatan_kontras(image):

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    olah_gambar = clahe.apply(image)
    return olah_gambar

def preprocessing_in_folder(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(('jpg', 'jpeg', '.png')):
                image_path = os.path.join(root, file)
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                if image is not None:
                    olah_gambar = peningkatan_kontras(image)
                    cv2.imwrite(image_path, olah_gambar)
                    print(f"memproses peningkatan kontras untuk {image_path}")
                else:
                    print(f"tidak dapat memproses {image_path}")

print("preprocessing di folder train ......")
preprocessing_in_folder(train_folder)

print("preprocessing di folder validation ......")
preprocessing_in_folder(validation_folder)

print("Preprocessing selesai")

preprocessing di folder train ......
memproses peningkatan kontras untuk images/train/surprise/39.jpg
memproses peningkatan kontras untuk images/train/surprise/8964.jpg
memproses peningkatan kontras untuk images/train/surprise/28326.jpg
memproses peningkatan kontras untuk images/train/surprise/28068.jpg
memproses peningkatan kontras untuk images/train/surprise/35076.jpg
memproses peningkatan kontras untuk images/train/surprise/8869.jpg
memproses peningkatan kontras untuk images/train/surprise/12564.jpg
memproses peningkatan kontras untuk images/train/surprise/26444.jpg
memproses peningkatan kontras untuk images/train/surprise/3017.jpg
memproses peningkatan kontras untuk images/train/surprise/1807.jpg
memproses peningkatan kontras untuk images/train/surprise/23071.jpg
memproses peningkatan kontras untuk images/train/surprise/22017.jpg
memproses peningkatan kontras untuk images/train/surprise/7972.jpg
memproses peningkatan kontras untuk images/train/surprise/20950.jpg
memproses peningkat

In [11]:
# 3. Computer vision ( segmentasi )

train_folder = "images/train"
validation_folder = "images/validation"

def segment_objects(image):
    _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    segmented_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)
    return segmented_image

def segment(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Gagal membaca citra: {image_path}")
        return

    segmented_image = segment_objects(image)
    return segmented_image

print("Segmentasi folder train ......")
for root, dirs, files in os.walk(train_folder):
    for file in files:
        if file.endswith(('jpg', 'jpeg', '.png')):
            image_path = os.path.join(root, file)
            segmented_image = segment(image_path)

            cv2.imwrite(image_path, segmented_image)

print("Segmentasi folder validation ......")
for root, dirs, files in os.walk(validation_folder):
    for file in files:
        if file.endswith(('jpg', 'jpeg', '.png')):
            image_path = os.path.join(root, file)
            segmented_image = segment(image_path)

            cv2.imwrite(image_path, segmented_image)

print("Segmentasi selesai")

Segmentasi folder train ......
Segmentasi folder validation ......
Segmentasi selesai


In [12]:
# 4. Teknik Deteksi Fitur ( SIFT ( Scale-Invariant Feature Transform ))

train_folder = "images/train"
validation_folder = "images/validation"

def detect_and_compute_sift(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Gagal membaca citra: {image_path}")
        return None


    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)

    return keypoints, descriptors

print("Ekstraksi fitur SIFT untuk folder train ......")
for root, dirs, files in os.walk(train_folder):
    for file in files:
        if file.endswith(('jpg', 'jpeg', '.png')):
            image_path = os.path.join(root, file)
            keypoints, descriptors = detect_and_compute_sift(image_path)

print("Ekstraksi fitur SIFT untuk folder validation ......")
for root, dirs, files in os.walk(validation_folder):
    for file in files:
        if file.endswith(('jpg', 'jpeg', '.png')):
            image_path = os.path.join(root, file)
            keypoints, descriptors = detect_and_compute_sift(image_path)

print ( "penggunaan teknik deteksi Fitur selesai")

Ekstraksi fitur SIFT untuk folder train ......
Ekstraksi fitur SIFT untuk folder validation ......
penggunaan teknik deteksi Fitur selesai


In [13]:
# 5. Konvolusi menghilangkan noise  ( Gaussian blur )

train_folder_path = "images/train"
validation_folder_path = "images/validation"

def apply_gaussian_blur(image_path, kernel_size=(5, 5)):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Gagal membaca citra: {image_path}")
        return None

    blurred_image = cv2.GaussianBlur(image, kernel_size, 0)
    return blurred_image

print("Penghilangan noise dengan filter Gaussian untuk folder train ......")
for root, dirs, files in os.walk(train_folder_path):
    for file in files:
        if file.endswith(('jpg', 'jpeg', '.png')):
            image_path = os.path.join(root, file)
            noise_reduced_image = apply_gaussian_blur(image_path)
            if noise_reduced_image is not None:
                cv2.imwrite(image_path, noise_reduced_image)

print("Penghilangan noise dengan filter Gaussian untuk folder validation ......")
for root, dirs, files in os.walk(validation_folder_path):
    for file in files:
        if file.endswith(('jpg', 'jpeg', '.png')):
            image_path = os.path.join(root, file)
            noise_reduced_image = apply_gaussian_blur(image_path)
            if noise_reduced_image is not None:
                cv2.imwrite(image_path, noise_reduced_image)

print("Penghilangan noise selesai")

Penghilangan noise dengan filter Gaussian untuk folder train ......
Penghilangan noise dengan filter Gaussian untuk folder validation ......
Penghilangan noise selesai
